In [2]:
import sim
import numpy as np
import math
import matplotlib.pyplot as plt
from compas_fab.robots import Robot

from compas.robots import RobotModel
from compas.robots import Joint
from compas.robots import Link

from compas.geometry import Frame
from compas.geometry import Transformation

In [3]:
def Rz(theta):

    return np.array([[  np.cos(theta), -np.sin(theta), 0 ],
                     [ np.sin(theta),  np.cos(theta), 0 ],
                     [ 0            ,  0            , 1 ]])

# Apenas para visualização de um referencial no plano
def plot_frame(Porg, R, c=['r', 'g']):

    axis_size = 3.0
    axes = axis_size*R

    x_axis = np.array(axes[0:2,0])
    y_axis = np.array(axes[0:2,1])

    # X
    plt.quiver(*Porg[:2], *x_axis, color=c[0], angles='xy', scale_units='xy', scale=1)

    # Y
    plt.quiver(*Porg[:2], *y_axis, color=c[1], angles='xy', scale_units='xy', scale=1)

In [35]:
print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19997,True,True,5000,5) # Connect to CoppeliaSim

if clientID!=-1:
    print ('Connected to remote API server')

    # Iniciando a simulação
    e = sim.simxStartSimulation(clientID,sim.simx_opmode_blocking)


    # variáveis auxiliares
    aux = np.array([0,0,0,1])
    aorg = np.array([0, 0])


    # Handle para o ROBÔ
    robotName = 'Pioneer_p3dx'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotName, sim.simx_opmode_oneshot_wait)

    # Posição e orientação do robô no mundo
    returnCode, posRobot = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
    returnCode, robotOrient = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)

    # Encontrando matriz de transformação Homogênea do universo para o robô
    pr_borg = np.array([posRobot[1], posRobot[2], 0])
    Rur = Rz(robotOrient[2])
    Tur = np.column_stack((Rur, pr_borg))
    Tur = np.row_stack((Tur, aux))

    print(Tur)

    # Encontrando matriz de transformação Homogênia do robô para o universo
    # Tru = np.linalg.inv(Tur)



    # Handle da cadeira
    chairName = 'conferenceChair'
    returnCode, chairHandle = sim.simxGetObjectHandle(clientID, chairName, sim.simx_opmode_oneshot_wait)

    # Posição e orientação da cadeira no mundo
    returnCode, posChair = sim.simxGetObjectPosition(clientID, chairHandle, -1, sim.simx_opmode_oneshot_wait)
    returnCode, chairOrient = sim.simxGetObjectOrientation(clientID, chairHandle, -1, sim.simx_opmode_oneshot_wait)

    posChair = np.append(posChair, 1)

    p0 = Tur @ posChair


    print(p0)
    print(sim.simxGetObjectPosition(clientID, objectHandle=robotHandle, relativeToObjectHandle=chairHandle, operationMode=sim.simx_opmode_oneshot_wait))


    # Plotando os referenciais

    # # Parando a simulação
    # sim.simxStopSimulation(clientID,sim.simx_opmode_blocking)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)

else:
    print ('Failed connecting to remote API server')

Program started
Connected to remote API server
[[ 0.81915085 -0.57357814  0.          2.17500186]
 [ 0.57357814  0.81915085  0.          0.13879307]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
[ 1.89917654 -0.20837618  0.43679929  1.        ]
(0, [-2.32019305229187, 2.505598783493042, 0.03058522939682007])
